In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
#training phase
num_features = 10 
input_shape = (num_features,)


input_data = tf.keras.Input(shape=input_shape)

hidden_layer = tf.keras.layers.Dense(units=64, activation=tf.nn.relu)(input_data)
output_layer = tf.keras.layers.Dense(units=num_features, activation=None)(hidden_layer)

model = tf.keras.Model(inputs=input_data, outputs=output_layer)

model.compile(optimizer='adam', loss='mean_squared_error')

num_samples = 1000
input_data = np.random.randn(num_samples, num_features)
output_data = input_data  

model.fit(input_data, output_data, epochs=100, batch_size=64)


In [ ]:
#save the model
model.save('my_trained_model.keras')

In [ ]:
#new data inserted here (testing phase)

model = tf.keras.models.load_model('my_trained_model.keras')

test_data = pd.read_csv('Testingdata01.csv')

test_data_np = test_data.to_numpy()

predictions = model.predict(test_data_np)

reconstruction_errors = np.mean(np.square(test_data_np - predictions), axis=1)

threshold = 0.1 

anomalies_indices = np.where(reconstruction_errors > threshold)[0]
print("Anomalies detected at indices:", anomalies_indices)